<a href="https://colab.research.google.com/github/vbetancurj/venpy/blob/master/NuevoHidro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pylab
import pandas as pd
from statsmodels.formula.api import ols
import statsmodels.api as sm
from matplotlib import pyplot
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator, AutoMinorLocator
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files
from pylab import *
from math import *
from datetime import datetime as dt
import datetime

In [ ]:
#Acceso a Drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Acceso archivo
#Estación 1
downloaded1 = drive.CreateFile({'id':'1Eb5pF4XgUlVGJPqRPZopGsPP_of7I3NH'})
downloaded1.GetContentFile('11130030.xlsx')

#Estación 2
downloaded2 = drive.CreateFile({'id':'1EbOxmi757INJjeRM3OzjgiI9n2oxVGhV'})
downloaded2.GetContentFile('11135010.xlsx')

#ONI
downloaded3 = drive.CreateFile({'id':'1MQjyYce8g079S-rN-VnlNvzA_o2L1li8'})
downloaded3.GetContentFile('ONI65_05.xlsx')
ONI= pd.read_excel('ONI65_05.xlsx')
ONI.index=ONI['Year']

In [ ]:
#Unir bases de datos
Datos1=pd.read_excel('11130030.xlsx')
Datos2=pd.read_excel('11135010.xlsx')
Datos2.index=[i for i in range(len(Datos1),len(Datos2)+len(Datos1))]
Datos=pd.concat([Datos1,Datos2], axis=0)

In [ ]:
#Convertir fechas a datetime
Años = [1969]
Años+= [i for i in range(1971,2006)]
Datos['Date'] = pd.to_datetime(Datos['Fecha'], format = '%Y/%m/%d')

#Agrega columnas de día del año, año, mes
Datos['DOY'] = Datos['Date'].dt.dayofyear
Datos['Año'] = Datos['Date'].dt.year
Datos['Mes'] = Datos['Date'].dt.month
Datos['Agrup'] = Datos['Mes']*0   #Variable para realizar agrupaciones
Datos['Año-mes'] = Datos['Date'].dt.strftime('%Y %m')
Datos['Año-mes2'] = pd.to_numeric(Datos['Date'].dt.strftime('%Y%m'))

In [ ]:
#Tablas
DatosMes=pd.pivot_table(Datos, index = "Mes", columns = 'Año', values = 'mm',aggfunc='sum')
DatosDia=pd.pivot_table(Datos, index = "DOY", columns = 'Año', values = 'mm',aggfunc='sum')

In [ ]:
#Fechas
fechas=[]
auxf=dt(2005,2,24)
fechai=dt(1969,1,1)
while fechai<auxf:
  fechas.append(fechai)
  fechai=fechai+datetime.timedelta(days=1)
fechas

In [ ]:
#Series de tiempo

AñosDescartados=[]
diasMes=[1,32,60,91,121,152,182,213,244,274,305,335,366]
tolerancia=4
plt.rcParams["figure.figsize"]=(10, 5)
fig, ax = plt.subplots()
DatosPromMeses=pd.DataFrame(data=[i for i in range(1,13)], index=[i for i in range(1,13)])
# for i in range(3):
#   ax[i].grid(axis='y')

auxindexaño=[]
auxtotaño=[]

for i in range(1969,2005):
  if i not in DatosDia:
    continue
  auxAño=DatosDia[i]
  auxindex=[]
  auxNumMes=[]
  auxtotmes=[]

  for k in range(12):
    auxMes=pd.DataFrame(data={"dias":[j for j in range(diasMes[k],diasMes[k+1])]},index=[j for j in range(diasMes[k],diasMes[k+1])])
    auxMes['mm']=auxAño
    if len(auxMes[auxMes['mm']>=0])+tolerancia>=len(auxMes):
      auxindex.append(dt(i,k+1,1))
      auxNumMes.append(k+1)
      auxtotmes.append(auxMes[auxMes['mm']>=0].sum())
  auxData=pd.DataFrame(data=auxtotmes,index=auxindex)
  auxData2=auxData['mm']
  auxData2.index=auxNumMes
  DatosPromMeses[i]=auxData2

  ax.set_ylabel('Precipitación (mm)', fontsize = 14)
  ax.tick_params(axis = 'x', which = 'major', labelsize = 12, width = 1.0, length = 7)
  ax.tick_params( axis = 'y', which = 'major', labelsize = 12,width = 1.0, length = 7)
  ax.yaxis.set_minor_locator(AutoMinorLocator(5))
  ax.xaxis.set_minor_locator(AutoMinorLocator(4))
  ax.tick_params(direction='out',which = 'minor', width = 1.0, length = 3)

  stemlines= ax.stem(auxData.index,auxData['mm'],markerfmt=' ',basefmt=" ")
  plt.setp(stemlines, 'linewidth', 1.8, color='dimgrey')

  plt.grid(axis='y')
  ax.set_ylim(0)


# plt.savefig('SerieMensual.png', bbox_inches='tight')
# files.download('SerieMensual.png')

In [ ]:
AñosDescartados=[]
diasMes=[1,32,60,91,121,152,182,213,244,274,305,335,366]
tolerancia=4
plt.rcParams["figure.figsize"]=(10, 5)
fig, ax = plt.subplots()
ax.set_ylabel('Precipitación (mm)', fontsize = 14)
DatosPromMeses=pd.DataFrame(data=[i for i in range(1,13)], index=[i for i in range(1,13)])
# for i in range(3):
#   ax[i].grid(axis='y')

auxindexaño=[]
auxtotaño=[]

for i in range(1969,2005):
  if i not in DatosDia:
    continue
  auxAño=DatosDia[i]
  auxindex=[]
  auxNumMes=[]
  auxtotmes=[]

  for k in range(12):
    auxMes=pd.DataFrame(data={"dias":[j for j in range(diasMes[k],diasMes[k+1])]},index=[j for j in range(diasMes[k],diasMes[k+1])])
    auxMes['mm']=auxAño
    if len(auxMes[auxMes['mm']>=0])+tolerancia>=len(auxMes):
      auxindex.append(dt(i,k+1,1))
      auxNumMes.append(k+1)
      auxtotmes.append(auxMes[auxMes['mm']>=0].sum())
  auxData=pd.DataFrame(data=auxtotmes,index=auxindex)
  auxData2=auxData['mm']
  auxData2.index=auxNumMes
  DatosPromMeses[i]=auxData2
  # print("Hay",len(auxData['dias']),"meses con datos en año",i)

  if len(auxindex)==12:
    # print('No se descarta',i)
    auxindexaño+=[i]
    auxtotaño+=[sum(auxData['mm'])]

  else:
    AñosDescartados.append(i)

  plt.bar(auxindexaño,auxtotaño, color='dimgrey')
  plt.xticks([1968,1972,1976,1980,1984,1988,1992,1996,2000,2004])
  ax.tick_params(axis = 'x', which = 'major', labelsize = 12, width = 1.0, length = 7)
  ax.tick_params(axis = 'y', which = 'major', labelsize = 12,width = 1.0, length = 7)
  ax.yaxis.set_minor_locator(AutoMinorLocator(5))
  ax.xaxis.set_minor_locator(AutoMinorLocator(4))
  ax.tick_params(direction='out',which = 'minor', width = 1.0, length = 3)
  plt.grid(axis='y', which='both')
  ax.set_axisbelow(True)
  ax.set_ylim(0)
  plt.yticks(range(0, 4500, 1000))

# plt.savefig('SerieAnual.png', bbox_inches='tight')
# files.download('SerieAnual.png')

  # stemlines= ax.stem(auxindexaño,auxtotaño,markerfmt=' ',basefmt=" ")
  # plt.setp(stemlines, 'linewidth', 1.8, color='dimgrey')

In [ ]:
# #Boxplot

DatosPromMeses=DatosPromMeses.transpose()
plt.rcParams["figure.figsize"]=(10, 5)
fig, ax = plt.subplots()
ax.grid(axis='y')
ax.set_ylabel('Precipitación (mm)', fontsize = 14)

# DatospromMes=pd.pivot_table(Datos, index = 'Año', columns = 'Mes', values = 'mm',aggfunc='sum')
meses=['Ene','Feb','Mar','Abr','May','Jun','Jul','Ago','Sep','Oct','Nov','Dic']

for i in range(1,13):
  aux=DatosPromMeses[DatosPromMeses[i]>=0][i][1:]
  # print(aux)
  ax.boxplot(aux,positions=[i-1],showfliers=False, showmeans = True, medianprops = dict(color = "black"), meanprops = dict(marker = "x", markersize = 6, markeredgecolor = "grey"), widths=0.6, whis=(0,100))
  plt.xticks(range(0, 12, 1), labels=meses, fontsize=14)

  ax.yaxis.set_minor_locator(AutoMinorLocator(4))
  ax.tick_params(direction='in',which = 'minor', width = 1.0, length = 3)
  plt.grid(axis='y')
  aux2=aux.mean(axis=0)
  # print(aux2)
auxprom=[53.02903225806452,38.39333333333333,
52.91481481481482,
137.02800000000002,
242.85384615384618,
285.70714285714286,
243.416,
239.67499999999995,
222.14230769230772,
237.13199999999998,
226.78214285714284,
120.324]
ax.plot(auxprom, color='slategrey', linewidth= 1, label='Media')
plt.legend(loc='upper left')

# plt.savefig('Boxplot.png', bbox_inches='tight')
# files.download('Boxplot.png')

In [ ]:
for i in range(1,13):
  aux=DatosPromMeses[DatosPromMeses[i]>=0][i][1:]
  auxprom=aux.mean(axis=0)
  print(auxprom)

In [ ]:
#ENSO

if 1969 not in AñosDescartados:
  AñosDescartados+=[1969]

#AñosDescartados2=[1969,1971]
AñosDescartados2=AñosDescartados
DatosPromMeses2=DatosPromMeses.transpose()

DatosPromMeses2=DatosPromMeses2.drop(0,axis=1)
for i in AñosDescartados2:
  DatosPromMeses2=DatosPromMeses2.drop(i,axis=1)

DatosPromMeses2=DatosPromMeses2.transpose()

ENSO=pd.DataFrame(Años)
ENSO.index=Años

x_niño,x_neut,x_niña=[],[],[] #niño,neutral,niña de cada mes
y_niño,y_neut,y_niña=[],[],[] #niño,neutral,niña de cada mes
meses=['Ene','Feb','Mar','Abr','May','Jun','Jul','Ago','Sep','Oct','Nov','Dic']

plt.rcParams["figure.figsize"]=(10, 7)
fig, ax = plt.subplots()
ax.set_ylabel('Precipitación (mm)', fontsize = 14)
#ax.set_title('ENSO (1971-2019): Estación Las Cintas (35190010)', fontsize = 15)
ax.tick_params(axis = 'x', which = 'major', labelsize = 14, width = 1.0, length = 5)
ax.tick_params(axis = 'y', which = 'major', labelsize = 14,width = 1.0, length = 5)
ax.tick_params(direction='in',which = 'minor', width = 1.0, length = 3)
# ax.grid(axis='y')
ax.grid(axis='y', which='both')


for i in range(0,60,5):
  auxmes=i//5
  auxONI=ONI[meses[auxmes]]
  DataAux=pd.DataFrame(data=DatosPromMeses2[auxmes+1])
  DataAux=DataAux[DataAux[auxmes+1]>=0]
  general=mean(DataAux[auxmes+1])

  stemlines= ax.stem([i+2],[general],markerfmt=' ',basefmt=" ")
  plt.setp(stemlines, 'color', 'lightgrey',linewidth=35)

  niño=DataAux[(auxONI=='niño')==True][auxmes+1]
  niña=DataAux[(auxONI=='niña')==True][auxmes+1]
  neut=DataAux[(auxONI=='neutral')==True][auxmes+1]
  niñom,niñam,neutm=niño.mean(),niña.mean(),neut.mean()
  x_niño+=[i+3.2]
  y_niño+=[niñom]
  x_neut+=[i+2]
  y_neut+=[neutm]
  x_niña+=[i+0.8]
  y_niña+=[niñam]

ax.plot(x_niño,y_niño,linewidth=1,color='red')
ax.plot(x_neut,y_neut,linewidth=1,color='dimgrey')
ax.plot(x_niña,y_niña,linewidth=1,color='blue')


for i in range(0,60,5):
  auxmes=i//5
  auxONI=ONI[meses[auxmes]]
  DataAux=pd.DataFrame(data=DatosPromMeses2[auxmes+1])
  DataAux=DataAux[DataAux[auxmes+1]>=0]

  ENSO['niño'+meses[auxmes]]=niño=DataAux[(auxONI=='niño')==True][auxmes+1]
  ENSO['niña'+meses[auxmes]]=niña=DataAux[(auxONI=='niña')==True][auxmes+1]
  ENSO['neut'+meses[auxmes]]=neut=DataAux[(auxONI=='neutral')==True][auxmes+1]

  niño=niño[niño>=0]
  niña=niña[niña>=0]
  neut=neut[neut>=0]

  niñom,niñam,neutm=niño.mean(),niña.mean(),neut.mean()
  niñomax,niñamax,neutmax=max(niño),max(niña),max(neut)
  niñomin,niñamin,neutmin=min(niño),min(niña),min(neut)

  ax.plot(i+3.2,niñom,'x',color='red')
  ax.plot(i+2,neutm,'o',color='dimgray',markersize=4)
  ax.plot(i+0.8,niñam,'*',color='blue')

  #Dispersión

  # print(niña)
  # print()
  # print('niña',meses[auxmes],niñamax,niñamin)
  # print()
  # print(neut)
  # print()
  # print('neut',meses[auxmes],neutmax,neutmin)
  # print()
  # print(niño)
  # print()
  # print('niño',meses[auxmes],niñomax,niñomin)
  # print()
  ax.plot(i+3.2,niñomax,'_',color='red')
  ax.plot(i+2,neutmax,'_',color='dimgray')
  ax.plot(i+0.8,niñamax,'_',color='blue')

  ax.plot(i+3.2,niñomin,'_',color='red')
  ax.plot(i+2,neutmin,'_',color='dimgray')
  ax.plot(i+0.8,niñamin,'_',color='blue')

  ax.plot([i+3.2,i+3.2],[niñomin,niñomax],color='red',linewidth=0.5)
  ax.plot([i+2,i+2],[neutmin,neutmax],color='dimgray',linewidth=0.5)
  ax.plot([i+0.8,i+0.8],[niñamin,niñamax],color='blue',linewidth=0.5)

plt.xticks([i for i in range(2,60,5)], meses)

ax.minorticks_on()
ax.xaxis.set_tick_params(which='minor', bottom=False)

#Leyendas
plt.plot([1,2],[-100,-99],color='lightgrey',linewidth=7)
plt.plot(1,-100,'x',label='El Niño',color='red')
plt.plot(1,-100,'o',label='Neutral',color='dimgray',markersize=4)
plt.plot(1, -100,'v',label='La Niña',color='blue')

plt.legend(loc='lower center', ncol= 3)
plt.ylim(-10)
plt.minorticks_on()

# plt.savefig('ENSO.png', bbox_inches='tight')
# files.download('ENSO.png')

plt.show()